In [1]:
#@title Install dependencies (takes ~1-2 minutes)
!pip -q install --upgrade pip
!pip -q install transformers accelerate sentencepiece
!pip -q install timm pillow einops
!pip -q install faster-whisper
!pip -q install opencv-python
!apt -qq install ffmpeg

import torch, platform
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available(), "| Device:", "cuda" if torch.cuda.is_available() else "cpu")
print("Python:", platform.python_version())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.8 MB/s eta 0:00:00
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Torch: 2.9.0+cu126 | CUDA available: True | Device: cuda
Python: 3.12.12


In [9]:
#@title 🎨 WhatsApp-Style Multimodal AI Chat (Clean Install)
# Restart runtime after running this cell
!pip install -q --force-reinstall gradio==3.50.2 huggingface_hub==0.19.4
!pip install -q faster-whisper opencv-python-headless transformers accelerate bitsandbytes
print("⚠️ RESTART RUNTIME NOW (Runtime > Restart runtime)")
print("Then run the cell below ⬇️")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
opencv-python 4.13.0.90 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.13.0.90 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.13.0.90 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
dataproc-spark-connect 1.0.1 requires websockets>=14.0, but you have websockets 11.0.3 which is in

In [4]:
#@title 🎨 AI Assitant
import torch
import gradio as gr
import cv2
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM, BlipProcessor, BlipForConditionalGeneration
from faster_whisper import WhisperModel
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else torch.float32

print("🔄 Loading models...")

tok = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
text_model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True
).to(device)

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base",
    torch_dtype=torch_dtype
).to(device)

whisper = WhisperModel("tiny", device=device, compute_type="int8")

print("✅ Models loaded!\n")

def chat_text(prompt, max_tokens=250):
    messages = [{"role": "user", "content": prompt}]
    text = tok.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    inputs = tok(text, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        out = text_model.generate(
            inputs,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2
        )
    full_text = tok.decode(out[0], skip_special_tokens=True)
    if "<|assistant|>" in full_text:
        return full_text.split("<|assistant|>")[-1].strip()
    elif "assistant\n" in full_text:
        return full_text.split("assistant\n")[-1].strip()
    return full_text.strip()

def caption_image(image):
    inputs = blip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        out = blip_model.generate(**inputs, max_new_tokens=50)
    return blip_processor.decode(out[0], skip_special_tokens=True)

def process_video(video_path, num_samples=10):
    cap = cv2.VideoCapture(video_path)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = num_frames / fps if fps > 0 else 0

    if num_frames == 0:
        cap.release()
        return "⚠️ Could not read video"

    # Sample evenly across video
    if num_frames <= num_samples:
        indices = list(range(num_frames))
    else:
        step = num_frames / (num_samples + 1)
        indices = [int(step * (i + 1)) for i in range(num_samples)]

    captions = []

    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = cap.read()
        if ok:
            img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            inputs = blip_processor(images=img, return_tensors="pt").to(device)
            with torch.no_grad():
                out = blip_model.generate(**inputs, max_new_tokens=60)
            caption = blip_processor.decode(out[0], skip_special_tokens=True)
            timestamp = idx / fps if fps > 0 else idx
            captions.append(f"⏱️ {timestamp:.1f}s: {caption}")

    cap.release()

    if not captions:
        return "⚠️ No frames processed"

    # NO LLM SUMMARY - just show the actual captions
    frame_list = "\n".join(captions)

    # Simple text summary based on captions (no AI generation)
    unique_descriptions = set([c.split(": ", 1)[1] if ": " in c else c for c in captions])
    simple_summary = f"Video shows: {', '.join(list(unique_descriptions)[:3])}"

    return f"""📹 **Video Analysis**

**Duration:** {duration:.1f} seconds
**Frames analyzed:** {len(captions)}

**What I see in each frame:**
{frame_list}

**Simple summary:** {simple_summary}"""

def transcribe_audio(audio_path):
    segments, info = whisper.transcribe(audio_path, beam_size=3)
    transcript = " ".join([s.text for s in segments])
    if not transcript.strip(): return "⚠️ No speech detected"

    # Simple word count summary instead of LLM
    word_count = len(transcript.split())
    return f"""🎤 **Audio Transcription**

**Language:** {info.language}
**Duration:** ~{word_count // 2} seconds (estimated)

**Full transcript:**
{transcript}"""

def respond(message, history, image, video, audio):
    try:
        if image:
            caption = caption_image(image)
            response = f"🖼️ **Image:** {caption}"
        elif video:
            response = process_video(video)
        elif audio:
            response = transcribe_audio(audio)
        elif message.strip():
            response = chat_text(message, max_tokens=300)
        else:
            response = "Send a message or upload media 📎"
    except Exception as e:
        response = f"⚠️ Error: {str(e)}"

    history.append((message if message else "[Media]", response))
    return "", history, None, None, None

css = """
.gradio-container {background: linear-gradient(135deg, #667eea, #764ba2); font-family: sans-serif;}
#chatbot {background: #e5ddd5; border-radius: 10px; box-shadow: 0 4px 15px rgba(0,0,0,0.2);}
"""

with gr.Blocks(css=css) as demo:
    gr.Markdown("# 💬 AI Assistant\n*Text • Image • Video • Audio*")
    chatbot = gr.Chatbot(elem_id="chatbot", height=500)
    with gr.Row():
        msg = gr.Textbox(placeholder="Type a message...", scale=7, show_label=False)
        send = gr.Button("Send", scale=1, variant="primary")
    with gr.Row():
        img = gr.Image(type="pil", label="📷 Image", scale=1)
        vid = gr.Video(label="🎥 Video", scale=1)
        aud = gr.Audio(type="filepath", label="🎤 Audio", scale=1)
    clear = gr.Button("🗑️ Clear")

    msg.submit(respond, [msg, chatbot, img, vid, aud], [msg, chatbot, img, vid, aud])
    send.click(respond, [msg, chatbot, img, vid, aud], [msg, chatbot, img, vid, aud])
    clear.click(lambda: ([], None, None, None), None, [chatbot, img, vid, aud])

demo.launch(debug=True, share=True)

🔄 Loading models...


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/473 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie text_decoder.cls.predictions.bias to text_decoder.cls.predictions.decoder.bias, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie text_decoder.bert.embeddings.word_embeddings.weight to text_decoder.cls.predictions.decoder.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
BlipForConditionalGeneration LOAD REPORT from: Salesforce/blip-image-captioning-base
Key                                       | Status     |  | 
------------------------------------------+------------+--+-
text_decoder.bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identic

✅ Models loaded!

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://7632d7dd599ae901ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7632d7dd599ae901ad.gradio.live
